# 1_환경 설정
- 필요한 라이브러리 로드 (pandas, numpy, os, re)  
- 구글 드라이브 연동 (구글 코랩에서 데이터 파일에 접근하기 위함)  

In [1]:
import pandas as pd
import numpy as np
import os
import sys
import re

# 2_데이터 불러오기

In [5]:
# 설정된 경로에서 엑셀 파일 불러오기
rawdata = 'data/A1_raw5.xlsx'

In [6]:
# 데이터 불러오기

df = pd.read_excel(rawdata)
print(df.shape)

df1 = df[:30] # 기사 30개만 샘플로
print(df1.shape)
df1[:3]

(5, 8)
(5, 8)


,자료ID,제목,저자,주제,발행일,URL,year,Content
0,ma_013_0010_0001,謝告,NaN,사고·편집후기,1920-06-25,http://db.history.go.kr/id/ma_013_0010_0001,1920,"謝告\n本 開闢 創刊號 中力萬能主義의 題末에 「금쌀악, 옥가루」와 檀君神話의 末段 ..."
1,ma_013_0010_0010,권두시,NaN,시,1920-06-25,http://db.history.go.kr/id/ma_013_0010_0010,1920,아- 風雲! 아- 霹靂!!\n모래가 날리며 돍이 닷도다.\n나무가 부러지며 풀이 쓸...
2,ma_013_0010_0020,創刊辭,NaN,사고·편집후기,1920-06-25,http://db.history.go.kr/id/ma_013_0010_0020,1920,創刊辭\n소리-있어 넓히 世界에 傳하니 온 世界 모든 人類-이에 應하야 부르짖기를 ...


# 3_기사를 문장별로 분리

In [7]:
# 기사를 문장별로 분리하는 사용자 함수
separators = ['\n', '.', '!', '?'] # 엔터 및 문장 종결 부호

def split_content_by_separators(content, separators):
    """Splits content based on the given separators."""
    escaped_separators = [re.escape(sep) for sep in separators]  # separators = 구분자들
    return re.split('|'.join(escaped_separators), content)

In [8]:
# 사용자 함수 적용하여 문장별로 분리하여 행으로 재구성

new_df = df1.apply(lambda row: pd.Series(split_content_by_separators(row['Content'], separators)), axis=1) \
            .stack() \
            .reset_index(level=1, drop=True) \
            .to_frame(name='Content') \
            .join(df1[['자료ID']], how='left')

new_df = new_df[['자료ID', 'Content']]
print(new_df.shape)
new_df

(320, 2)


,자료ID,Content
0,ma_013_0010_0001,謝告
0,ma_013_0010_0001,"本 開闢 創刊號 中力萬能主義의 題末에 「금쌀악, 옥가루」와 檀君神話의 末段 二行과 ..."
0,ma_013_0010_0001,
0,ma_013_0010_0001,開闢編輯室
0,ma_013_0010_0001,正誤
...,...,...
4,ma_013_0010_0040,故로 老年이 靑年을 蔑視하는 一事는 오로지 輕佻浮薄을 問題로 삼는 바라
4,ma_013_0010_0040,故로 靑年된 者-이 點에서 自重自愼하야 勇斷이 잇서도 輕擧에 失치 안이하도록 經驗...
4,ma_013_0010_0040,
4,ma_013_0010_0040,吾人은 어대까지든지 各地 靑年會의 勃興을 歡迎하며 그리하야 그의 自覺이 잇기를 바라...


In [9]:
# 문장별 id 생성 ##### 중요 !!
new_sent_df = new_df.reset_index(drop=True)
new_sent_df['sent_id'] = new_sent_df.index + 1
new_sent_df = new_sent_df[['sent_id', '자료ID', 'Content']]
new_sent_df

,sent_id,자료ID,Content
0,1,ma_013_0010_0001,謝告
1,2,ma_013_0010_0001,"本 開闢 創刊號 中力萬能主義의 題末에 「금쌀악, 옥가루」와 檀君神話의 末段 二行과 ..."
2,3,ma_013_0010_0001,
3,4,ma_013_0010_0001,開闢編輯室
4,5,ma_013_0010_0001,正誤
...,...,...,...
315,316,ma_013_0010_0040,故로 老年이 靑年을 蔑視하는 一事는 오로지 輕佻浮薄을 問題로 삼는 바라
316,317,ma_013_0010_0040,故로 靑年된 者-이 點에서 自重自愼하야 勇斷이 잇서도 輕擧에 失치 안이하도록 經驗...
317,318,ma_013_0010_0040,
318,319,ma_013_0010_0040,吾人은 어대까지든지 各地 靑年會의 勃興을 歡迎하며 그리하야 그의 自覺이 잇기를 바라...


# 4_형태소 분석
- 제목: 형태소 분석기 적용
- kiwi 형태소 분석기로 텍스트 데이터 분석
- 한국어 형태소 분석기(kiwi)에서 제공하는 서브워드(subword) 학습 함수를 활용해 1920~40년에 출간된 조선/동아일보 약 190만건의 기사를 학습 후 만든 근대 한국어 토크나이저.
https://github.com/ByungjunKim/ModernKoreanSubword
- cf) 유튜브 동영상 https://www.youtube.com/watch?v=5VmKXYwnYeQ


## kiwi 설치

In [10]:
# kiwi 설치
!pip install -U kiwipiepy

In [11]:
# 서브워드 토크나이저 json 파일 다운로드
!git clone https://github.com/ByungjunKim/ModernKoreanSubword

fatal: destination path 'ModernKoreanSubword' already exists and is not an empty directory.


In [12]:
# kiwi 로드
from kiwipiepy import Kiwi
kiwi = Kiwi()
from kiwipiepy.sw_tokenizer import SwTokenizer

# 토크나이저 설정
tokenizer = SwTokenizer('./ModernKoreanSubword/ModernKoreanSubword.json', kiwi=kiwi)

## 형태소 분석 함수 작성

In [13]:
# 1음절 한자 이어질 때 두 개를 합해 줌

def is_hanja(ch):
    # 한자의 범위는 U+4E00 부터 U+9FFF 입니다.
    return '\u4e00' <= ch <= '\u9fff'

def combine_one_hanja(lst):
    result = []
    i = 0
    while i < len(lst):
        if (len(lst[i]) == 1 and i + 1 < len(lst) and len(lst[i + 1]) == 1 and
        is_hanja(lst[i]) and is_hanja(lst[i + 1])):
            result.append(lst[i] + lst[i + 1])
            i += 2
        else:
            result.append(lst[i])
            i += 1
    return result

In [14]:
# 영어, 숫자, 특정 부호 등 제외하기

def filter_elements(lst):
    # 영어, 숫자 및 '<' 문자만을 허용하는 정규식 패턴
    pattern = re.compile(r'^[a-zA-Z0-9<＞「」]+$')

    # 해당 패턴과 일치하지 않는 문자열만을 반환하는 필터링 함수
    return list(filter(lambda x: not pattern.match(x), lst))

In [15]:
# 본격 형태소 분석

import string
def do_morph(sent):
    # 토큰나이징 결과 확인 (subword + 한국어 형태소)
    vocab_list = tokenizer.encode(sent)
    result01 = [tokenizer.id2vocab[i].replace('#', '') for i in vocab_list if '/' not in tokenizer.id2vocab[i]]
    result01 = [''.join([char for char in word if char not in string.punctuation]) for word in result01]
    result01 = [word for word in result01 if len(word) > 0] # 공백 제외
    result01 = combine_one_hanja(result01) # 연속되는 1음절 한자 합침 (사용자 함수)
    result01 = filter_elements(result01) # 영어, 숫자, 특정부호 제외 (사용자 함수)

    return result01

## 형태소 분석 및 정리 정돈

In [16]:
new_sent_df[:3]

,sent_id,자료ID,Content
0,1,ma_013_0010_0001,謝告
1,2,ma_013_0010_0001,"本 開闢 創刊號 中力萬能主義의 題末에 「금쌀악, 옥가루」와 檀君神話의 末段 二行과 ..."
2,3,ma_013_0010_0001,


In [17]:
# 'Content' 열에 kiwi 형태소 분석기 적용
new_sent_df['tokens'] = new_sent_df['Content'].apply(do_morph)
new_sent_df

,sent_id,자료ID,Content,tokens
0,1,ma_013_0010_0001,謝告,[謝告]
1,2,ma_013_0010_0001,"本 開闢 創刊號 中力萬能主義의 題末에 「금쌀악, 옥가루」와 檀君神話의 末段 二行과 ...","[本, 開闢, 創刊, 號中, 力, 萬能, 主義, 題末, 금, 쌀, 악, 옥, 가루,..."
2,3,ma_013_0010_0001,,[]
3,4,ma_013_0010_0001,開闢編輯室,"[開闢, 編輯, 室]"
4,5,ma_013_0010_0001,正誤,[正誤]
...,...,...,...,...
315,316,ma_013_0010_0040,故로 老年이 靑年을 蔑視하는 一事는 오로지 輕佻浮薄을 問題로 삼는 바라,"[故老, 年, 靑年, 蔑視, 一事, 오로지, 輕佻, 浮薄, 問題, 바]"
316,317,ma_013_0010_0040,故로 靑年된 者-이 點에서 自重自愼하야 勇斷이 잇서도 輕擧에 失치 안이하도록 經驗...,"[故, 靑年, 者, 이, 點, 自重, 自愼, 勇斷, 잇서, 도, 輕擧, 失, 經驗,..."
317,318,ma_013_0010_0040,,[]
318,319,ma_013_0010_0040,吾人은 어대까지든지 各地 靑年會의 勃興을 歡迎하며 그리하야 그의 自覺이 잇기를 바라...,"[吾人, 어대, 各地, 靑年會, 勃興, 歡迎, 그리, 그, 自覺, 前進, 事業, 好..."


In [18]:
# 'tokens' 열에서 리스트 요소가 하나도 없는 행 제거
new_sent_df1 = new_sent_df[new_sent_df['tokens'].str.len() > 0]

new_sent_df1 = new_sent_df1.copy()
new_sent_df1['tokens'] = new_sent_df1['tokens'].apply(lambda x: ' '.join(x)) # 리스트를 문자열로

new_sent_df1.reset_index(drop=True, inplace=True) # 인덱스 초기화
new_sent_df1['sent_id'] = new_sent_df1.index + 1 # sent_id 재넘버링
new_sent_df1.head()

,sent_id,자료ID,Content,tokens
0,1,ma_013_0010_0001,謝告,謝告
1,2,ma_013_0010_0001,"本 開闢 創刊號 中力萬能主義의 題末에 「금쌀악, 옥가루」와 檀君神話의 末段 二行과 ...",本 開闢 創刊 號中 力 萬能 主義 題末 금 쌀 악 옥 가루 檀君 神話 末段 二行 그...
2,3,ma_013_0010_0001,開闢編輯室,開闢 編輯 室
3,4,ma_013_0010_0001,正誤,正誤
4,5,ma_013_0010_0001,本號外 一三八頁 「平濟塔記事中」興王寺는 王興寺로,本 號外 一三八 頁 平濟 塔 記事中 興王 寺王 興寺


# 5_[활용] 단어 빈도

In [19]:
# 'tokens' 열의 문자열을 빈칸으로 분리하여 모든 단어들을 리스트로 추출
words = new_sent_df1['tokens'].str.split().explode()
words

0      謝告
1       本
1      開闢
1      創刊
1      號中
       ..
243    事業
243    好果
243    바로
243     다
243    未完
Name: tokens, Length: 3606, dtype: object

In [20]:
# 단어들의 빈도 계산
word_counts = words.value_counts()

# 결과 출력
print(word_counts[:20])

tokens
世界    79
이     78
그     56
것     53
안     47
우리    39
다     37
新     25
宗敎    24
自己    23
바     21
人類    20
朝鮮    20
信仰    20
者     20
吾人    19
을     18
안이    18
그리    17
活動    17
Name: count, dtype: int64


In [21]:
# 인덱스(단어)의 길이가 2 이상인 것만 선택
word_counts_1 = word_counts[word_counts.index.str.len() >= 2]
word_counts_1[:20]

tokens
世界    79
우리    39
宗敎    24
自己    23
人類    20
朝鮮    20
信仰    20
吾人    19
안이    18
그리    17
活動    17
무엇    15
소리    15
今日    15
人民    15
開闢    15
國家    14
進化    13
가티    13
現象    12
Name: count, dtype: int64

# The End of Notebook